# Kaczmarz Method(s)

In [13]:
using LinearAlgebra
n, m = 10, 4
A = randn(n,m)
x = randn(m)
display(x)
b = A*x
norm(A\b -x)

4-element Array{Float64,1}:
  0.49079115665873596
  0.36605810351764684
 -0.25998470633963633
 -0.377332333672407  

3.090730095650652e-16

In [15]:
x0 = zeros(size(x));
x1 = copy(x0);

The Kaczmarz update step is:
$$x_{k+1}=x_k+\frac{(b_j - A[j,:]'*x_k}{\|A[j,:]\|^2}*A[j,:]$$
where 
$j \in {1, ..., n}$ 
is a row number.

## Cyclical Kaczmarz

In [16]:
for j in 1:(n*100)
    i = mod1(j,n)
#    println(j)
    x1 = x0+((b[i] - A[i,:]'*x0)/norm(A[i,:])^2)*A[i,:]
    x0 = copy(x1)
#    println(x1)
end
x1

4-element Array{Float64,1}:
  0.490791156658736  
  0.36605810351764684
 -0.25998470633963633
 -0.37733233367240704

## Random Permutation Kaczmarz

In [17]:
using Random
x0=zeros(size(x))
norms=zeros(1,100)
for k in 1:100
    B = A[randperm(n),:]
    for i in 1:n
        x1 = x0+((b[i] - B[i,:]'*x0)/norm(B[i,:])^2)*B[i,:]
        x0 = copy(x1)
    end
    norms[k] = norm(x-x1)
end
x1

4-element Array{Float64,1}:
  0.29747942473191047
  0.3766460747269807 
 -0.5874133576138836 
 -0.5416332304392768 